In [1]:
# imports
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


# Create ontology

In [2]:
# create new ontology
game_ontology = get_ontology('http://test.org/game_ontology.owl')
game_ontology.destroy()
game_ontology = get_ontology('http://test.org/game_ontology.owl')

## Atomic concepts and their properties

In [3]:
with game_ontology:
    # Classes (1 - 5)
    class Game(Thing):
        pass
    class Field(Thing):
        pass
    class Equipment(Thing):
        pass    
    class Spectator(Thing):
        pass
    class Player(Thing):
        pass

    # Properties (1 - 11)
    class played_on(Game >> Field):
        pass

    class require(Game >> Equipment):
        pass

    class has_duration(Game >> int):
        pass

    class is_professional(Game >> bool, FunctionalProperty):
        pass
    class is_amateur(Game >> bool, FunctionalProperty):
        Not(is_professional)
        pass

    class plays(Player >> Game):
        pass
    class played_by(Game >> Player):
        inverse_property = plays
        pass

    class watches(Spectator >> Game):
        pass
    class watched_by(Game >> Spectator):
        inverse_property = watches
        pass
    class has_interest_in(Spectator >> Game):
        pass
    class has_to_remain_silent(Spectator >> bool, FunctionalProperty):
        pass

## Composed concepts and their properties

### Fields

In [4]:
with game_ontology:
    # Classes (6 - 7)
    class Lawn(Field):
        pass
    class Board(Field):
        pass    

    # Properties (12 - 12)
    class is_custom_board(Board >> bool, FunctionalProperty):
        pass

### Equipments

In [5]:
with game_ontology:
    # Classes (8 - 16)
    class Table(Equipment):
        pass
    class Card(Equipment):
        pass
    class Dice(Equipment):
        pass
    class Pawn(Equipment):
        pass
    class Paper(Equipment):
        pass
    class Pencil(Equipment):
        pass
    class Computer(Equipment):
        pass
    class SportEquipment(Equipment):
        pass
    class Ball(SportEquipment):
        pass

    # Properties (13 - 20)
    class is_normal_card(Card >> bool, FunctionalProperty):
        pass
    class is_custom_card(Card >> bool, FunctionalProperty):
        Not(is_normal_card)
        pass

    class has_sides(Dice >> int, FunctionalProperty):
        pass
    class has_number(Dice >> int, FunctionalProperty):
        pass
    class has_greater_number_than(Dice >> Dice, TransitiveProperty):
        pass
    class has_lower_number_than(Dice >> Dice, TransitiveProperty):
        inverse_property = has_greater_number_than
        pass
    
    class is_blank(Paper >> bool, FunctionalProperty):
        pass
    class has_color(Pencil >> str):
        pass

### Spectators

In [6]:
with game_ontology:
    # Classes (17 - 18)
    class Fan(Spectator):
        equivalent_to = [has_interest_in.min(1, Game)]
        pass
    class Kibitzer(Spectator):
        equivalent_to = [watches.min(1, Game)
                        & has_to_remain_silent.value(True)]
        pass

    # Properties (21 - 22)
    class has_favourite(Fan >> Player):
        pass
    class has_fans(Player >> Fan):
        inverse_property = has_favourite
        pass

### Games by number of players

In [7]:
with game_ontology:
    # Classes (19 -20)
    class SinglePlayerGame(Game):
        equivalent_to = [played_by.exactly(1, Player)] 
        pass
    class MultiPlayerGame(Game):
        equivalent_to = [played_by.min(2, Player)]
        pass
    
    # Properties (23 - 25)
    class has_players_number(MultiPlayerGame >> int):
        pass
    class has_team(MultiPlayerGame >> bool, FunctionalProperty):
        pass
    class has_opponent(MultiPlayerGame >> bool, FunctionalProperty):
        pass

### Games by activity

In [8]:
with game_ontology:
    # Classes (21 - 25)
    class IntelectualGame(Game):
        pass
    class LogicalGame(IntelectualGame):
        pass
    class StrategyGame(IntelectualGame):
        pass
    class KnowledgeGame(IntelectualGame):
        pass
    class PhysicalActivityGame(Game):
        pass
        
    # Properties (-)
    ...

### Sport Games

In [9]:
with game_ontology:
    # Classes (26 - 31)
    class TeamGame(MultiPlayerGame):
        is_a = [has_team.value(True)]      
        pass
    class SportGame(PhysicalActivityGame):
        equivalent_to = [require.only(SportEquipment)
                        & watched_by.some(Spectator)]
        pass
    class IndividualSportGame(SinglePlayerGame, SportGame):
        pass
    class TeamSportGame(TeamGame, SportGame):
        pass
    class ProfessionalSportGame(SportGame):
        is_a = [is_professional.value(True)] 
        pass 
    class ProfessionalBallGameOnLawn(ProfessionalSportGame):
        equivalent_to = [require.min(1, Ball)
                        & played_on.only(Lawn)]
        pass
    
    # Properties (26 - 26)
    class has_referee(ProfessionalSportGame >> bool, FunctionalProperty):
        pass

### Cards

In [10]:
with game_ontology:
    # Classes (32 - 35)
    class PlayingCard(Card):
        is_a = [is_normal_card.value(True)] 
        pass
    class MagicCard(Card):
        is_a = [is_custom_card.value(True)]
        pass

    class CardSuit(Thing):
        pass
    class CardValue(Thing):
        pass

    # Properties (27 - 32)    
    class has_suit(PlayingCard >> CardSuit, FunctionalProperty):
        pass
    class has_value(PlayingCard >> CardValue, FunctionalProperty):
        pass
    class has_greater_value_than(PlayingCard >> PlayingCard, TransitiveProperty):
        pass
    class has_lower_value_than(PlayingCard >> PlayingCard, TransitiveProperty):
        inverse_property = has_greater_value_than
        pass

    class has_type(MagicCard >> str, FunctionalProperty):
        pass
    class has_effect(MagicCard >> str):
        pass

suits = [CardSuit('Clubs'), CardSuit('Diamonds'), CardSuit('Hearts'), CardSuit('Spades')]
CardSuit.is_a.append(OneOf(suits))

values = [CardValue('Two'), CardValue('Three'), CardValue('Four'), CardValue('Five'),
          CardValue('Six'), CardValue('Seven'), CardValue('Eight'), CardValue('Nine'),
          CardValue('Ten'), CardValue('Jack'), CardValue('Queen'), CardValue('King'), CardValue('Ace')]
CardValue.is_a.append(OneOf(values))

### Table Games

In [11]:
with game_ontology:
    # Classes (36 - 41)
    class VideoGame(Game):
        equivalent_to = [require.min(1, Computer)]
        pass
    class TableTopGame(StrategyGame, LogicalGame):
        equivalent_to = [StrategyGame 
                        & LogicalGame
                        & require.min(1, Table)]
        pass
    class BoardGame(TableTopGame):
        equivalent_to = [played_on.min(1, Board)]
        pass
    class DiceGame(TableTopGame):
        equivalent_to = [require.some(Dice)]
        pass
    class PaperAndPencilGame(TableTopGame):
        equivalent_to = [require.some(Paper & Pencil)]
        pass
    class CardGame(TableTopGame):
        equivalent_to = [require.some(Card)]
        pass
    
    # Properties (33 - 33)
    class need(require):
        domain    = [BoardGame]
        range     = [MagicCard | Dice | Pawn | Paper | Pencil]
        pass

# Instances

In [12]:
# Instances
tennis_ball = Ball('tennis_ball')
tennis_court_clay = Field('tennis_court_clay')
tennis_court_hard = Field('tennis_court_hard')
tennis_racket = Equipment('tennis_racket')
tennis = ProfessionalSportGame('tennis', require = [tennis_ball, tennis_racket], played_on = [tennis_court_clay, tennis_court_hard])

green = Lawn('green')
golf_ball = Ball('golf_ball')
golf_stick = SportEquipment('golf_stick')
golf = ProfessionalBallGameOnLawn('golf', require = [golf_ball, golf_stick], played_on = [green])
golf.is_a.append(IndividualSportGame)

football = ProfessionalBallGameOnLawn('football', has_referee = True, has_duration = [90])
football.is_a.append(TeamSportGame)

desk = Table('desk')
laptop = Computer('laptop')
hardstone = VideoGame('Hardstone', require = [laptop, desk])

paper = Paper('paper', is_blank = True)
pencil = Pencil('pencil', has_color = ['red', 'blue'])
tic_tac_toe = PaperAndPencilGame('tic_tac_toe', require = [paper, pencil], is_amateur = True)

pawn = Pawn('pawn')
knight = Pawn('knight')
bishop = Pawn('bishop')
rook = Pawn('rook')
queen = Pawn('queen')
king = Pawn('king')
chess_board = Board('chess_board')
chess = BoardGame('chess', played_on = [chess_board], need = [pawn, knight, bishop, rook, queen, king])
chess.is_a.append(MultiPlayerGame)
chess.is_a.append(has_opponent.value(True))
chess.is_a.append(has_players_number.value(2))

who_wants_to_be_a_millionaire = KnowledgeGame('who_wants_to_be_a_millionaire')
who_wants_to_be_a_millionaire.is_a.append(SinglePlayerGame)

table = Table('table')
go_board = Board('go_board', is_custom_board = True)
go = IntelectualGame('go', has_players_number = [2], has_opponent = True, require = [table], played_on = [go_board])
go.is_a.append(LogicalGame)
go.is_a.append(StrategyGame)

North = Player('North', plays = [golf, football])
East = Player('East', plays = [hardstone])
South = Player('South')
West = Player('West')

kibitzer = Kibitzer('kibitzer', watches = [football], has_interest_in = [hardstone])
fan = Fan('fan', has_favourite = [East, West], watches = [tennis])
spectator = Spectator('spectator', has_interest_in = [football, chess])

deck = []
card = None
for suit in CardSuit.instances():
    prev_card = None
    for value in CardValue.instances():
        s = suit.get_name()
        v = value.get_name()
        if prev_card:
            card = PlayingCard(f'{v}_of_{s}', has_suit = suit, has_value = value, has_greater_value_than = [prev_card])
        else:
            card = PlayingCard(f'{v}_of_{s}', has_suit = suit, has_value = value)
        deck.append(card)
        prev_card = card

bridge = CardGame('Bridge', played_by = [North, East, South, West], has_team = True, watched_by = [fan, kibitzer], require = deck)

dice_6 = Dice('dice_6', has_sides = 6, has_number = 5) 
dice_12 = Dice('dice_12', has_sides = 12, has_number = 3, has_lower_number_than = [dice_6])
dice_8 = Dice('dice_8', has_sides = 8, has_number = 7, has_greater_number_than = [dice_6, dice_12])
backgammon = BoardGame('Backgammon', require = [dice_6], has_opponent = True)
backgammon.is_a.append(MultiPlayerGame)

catan = BoardGame('Catan', require = [MagicCard], has_team = False)       

### Subsumption examples

In [13]:
with game_ontology:
    class GameWithAtLeastFourPlayers(Game):            # GameWithAtLeastFourPlayers <= MultiPlayerGame
        equivalent_to = [played_by.min(4, Player)]
        pass
    class ComputerNetworkGame(Game):                   # ComputerNetworkGame <= VideoGame
        equivalent_to = [require.min(2, Computer)]
        pass

### Classification examples

In [14]:
with game_ontology:
    class OnlineMultiplayerCardGame(Game):             # Game -> Game & VideoGame & MultiPlayerGame & CardGame        
        equivalent_to = [require.min(1, Computer)
                        & played_by.min(2, Player)
                        & require.some(Card)]
        pass

    class GamblingGame(Game):                          # Game -> Game & BoardGame & CardGame & DiceGame
        equivalent_to = [require.some(PlayingCard)
                        & need.exactly(2, Dice)
                        & require.min(1, Table)]


# Test

In [15]:
def tests():
    # Spectators
    print(spectator.is_a)                              # Spectators -> Fan
    print(kibitzer.__class__)                          # Kibitzer -> Kibitzer & Fan

    # sport games
    print(tennis_racket.is_a)                          # Equipment -> SportEquipment
    print(golf.get_properties())                       # require & played on & played by
    print(football.INDIRECT_get_properties())          # professional & team & referee & duration & require & played by & watched by

    # board games
    print(chess.__class__)                             # BoardGame & MultiPlayerGame
    print(chess.is_instance_of)                        # BoardGame & MultiPlayerGame & has 2 players & has opponent
    print(chess.need)                                  # pawn & knight & bishop & rook & queen & king
    print(go.is_a)                                     # IntelectualGame & LogicalGame & StrategyGame -> BoardGame & MultiPlayerGame
    print(backgammon.is_a)                             # BoardGame & MultiPlayerGame -> BoardGame & MultiPlayerGame & DiceGame

    # card games
    print(bridge.played_by)                            # North & East & South & West
    print(bridge.is_a)                                 # CardGame -> CardGame & MultiPlayerGame
    print(catan.require)                               # MagicCards
    print(catan.get_properties())                      # has_team & require
    print(catan.is_a)                                  # BoardGame -> BoardGame & MultiPlayerGame

    # cards
    print(deck[12].has_greater_value_than)             # KS
    print(deck[12].INDIRECT_has_greater_value_than)    # All spades without AS - (12 cards)
    print(deck[19].INDIRECT_has_lower_value_than)      # 9H & 10H & JH & QH & KH & AH - (6 cards)

    # dices
    print(dice_12.has_lower_number_than)               # dice_6 & dice_8
    print(dice_12.has_greater_number_than)             # [] (empty list because dice_12 has smallest number)

tests()

[game_ontology.Spectator]
game_ontology.Kibitzer
[game_ontology.Equipment]
{game_ontology.require, game_ontology.played_by, game_ontology.played_on}
{game_ontology.is_professional, game_ontology.played_by, game_ontology.watched_by, game_ontology.has_referee, game_ontology.has_team, game_ontology.require, game_ontology.has_duration}
<FusionClass game_ontology.BoardGame, game_ontology.MultiPlayerGame>
[game_ontology.BoardGame, game_ontology.MultiPlayerGame, game_ontology.has_opponent.value(True), game_ontology.has_players_number.value(2)]
[game_ontology.pawn, game_ontology.knight, game_ontology.bishop, game_ontology.rook, game_ontology.queen, game_ontology.king]
[game_ontology.IntelectualGame, game_ontology.LogicalGame, game_ontology.StrategyGame]
[game_ontology.BoardGame, game_ontology.MultiPlayerGame]
[game_ontology.North, game_ontology.East, game_ontology.South, game_ontology.West]
[game_ontology.CardGame]
[game_ontology.MagicCard]
{game_ontology.has_team, game_ontology.require}
[game

### Reasoning

In [16]:
def reasoner():
    owlready2.JAVA_EXE = "C:\\Program Files\\java\\jdk-17\\bin\\java.exe"

    with game_ontology:
        sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

reasoner()

* Owlready2 * Running Pellet...
    C:\Program Files\java\jdk-17\bin\java.exe -Xmx2000M -cp C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\Tanya\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\Tanya\AppData\Loc

* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Ace_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Seven_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Ten_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Six_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Eight_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Nine_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.King_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Five_of_Hearts
* Owlready * Adding relation game_ontology.Three_of_Hearts has_lower_value_than game_ontology.Jack_of_Hearts
* Owlready * Adding 

* Owlready2 * Pellet took 8.709792613983154 seconds
* Owlready * Reparenting game_ontology.tennis_racket: {game_ontology.Equipment} => {game_ontology.SportEquipment}
* Owlready * Reparenting game_ontology.Backgammon: {game_ontology.BoardGame, game_ontology.MultiPlayerGame} => {game_ontology.DiceGame, game_ontology.MultiPlayerGame, game_ontology.BoardGame}
* Owlready * Reparenting game_ontology.go: {game_ontology.IntelectualGame, game_ontology.StrategyGame, game_ontology.LogicalGame} => {game_ontology.BoardGame, game_ontology.MultiPlayerGame}
* Owlready * Reparenting game_ontology.Catan: {game_ontology.BoardGame} => {game_ontology.BoardGame, game_ontology.MultiPlayerGame}
* Owlready * Reparenting game_ontology.GamblingGame: {game_ontology.Game} => {game_ontology.DiceGame, game_ontology.CardGame, game_ontology.BoardGame}
* Owlready * Reparenting game_ontology.Bridge: {game_ontology.CardGame} => {game_ontology.CardGame, game_ontology.MultiPlayerGame}
* Owlready * Reparenting game_ontology

# Save

In [17]:
game_ontology.save('../game_ontology.owl')